In [42]:
from collections import namedtuple, defaultdict
import numpy as np
from enum import Enum

In [98]:
class DataLine:
    def __init__(self, 
                 page_number: str=None, #:Nстраницы
                 line_number: str=None, #:Nстроки, сквозной
                 first_symbol: str=None, #:Nсимвола, начального, сквозной
                 int_from_prev: str=None, #:дельта-v-coord-up 
                 int_from_next: str=None, #:дельта-v-coord-down
                 h_left: str=None, #:h-coord начала
                 h_right: str=None, #:h-coord конца + 1
                 v: str=None, #:v-coord
                 font0_size: str=None, #:font-height0
                 font0_name: str='', #:font-name0
                 font1_size: str=None, #:font-height1
                 font1_name: str='', #:font-name1
                 text: str=None, #:TEXT
                ):
        self.page_number = int(page_number) if len(page_number) else 0
        self.line_number = int(line_number) if len(line_number) else 0
        self.first_symbol = int(first_symbol) if len(first_symbol) else 0
        
        self.int_from_prev = int(int_from_prev) if len(int_from_prev) else 0
        self.int_from_next = int(int_from_next) if len(int_from_next) else 0
        
        self.h_left = int(h_left) if len(h_left) else 0
        self.h_right = int(h_right) if len(h_right) else 0
        self.v = int(v) if len(v) else 0
        
        self.font0_size = int(font0_size) if len(font0_size) else 0
        self.font0_name = font0_name
        self.font1_size = int(font1_size) if len(font1_size) else 0
        self.font1_name = font1_name
        
        self.text = text
        
        self.label = '?'

In [99]:
structure_file_name = 'processed/aerospace/y:2015:i:1:p:31-57.json'

In [100]:
data = []

with open(structure_file_name, "r", encoding="utf8") as read_file:
    for line in read_file:
        splitted = line.replace('\n', '').split(':')
        #print(splitted, "LEN", len(splitted))

        if len(splitted) > 13:
            rt = ':'.join(splitted[12:])
            data_line = DataLine(*splitted[:12], rt)
        elif len(splitted) == 13:
            data_line = DataLine(*splitted)
        else:
            pass
            assert len(splitted) >= 12, 'must have at least 12 elements'
        data.append(data_line)


In [101]:
len(data)

975

**Notation**:

- A --- paragraph starts
- B --- paragraph continues
- C --- header starts
- D --- bukvitsa
- ? --- other

# Нахождение отступа основного текста

Основной текст (в одноколочных публикациях) должен находиться левее всего.

Рассмотрим массивы со значениями левого и правого отступов от краев страницы для всех строк документа и выделим m наиболее частотных (для обоих массивов). При этом число m можно выбирать, основываясь на статистике документа. Если в тексте n колонок, то должно быть подавляющее большинство строк, отступы которых попали в топ-m. 

Далее для каждой строки делаем так: если строка начинается в одном из топ-m левых значений AND заканчивается в одном из топ-m правых значений [и при этом еще, например, написала самым частым шрифтом], то это — середина абзаца. Если выполняется первое условие, а координаты правой части не попадают в топ-m, то это конец абзаца. 
Наконец, если левая граница не входит в топ-m левых, а правая входит в топ-m правых, то это — начало абзаца. 

Случай с буквицей можно обработать как-нибудь отдельно. Это более-менее общие правила, которые, как мне кажется, должны работать везде. 

Заголовки при таком подходе тоже определяются просто. Во-первых, они должны быть набраны не самым частым шрифтом, во-вторых, обычно отступ справа у них не попадает в топ-m.

In [102]:
import matplotlib.pyplot as plt
%matplotlib inline

In [103]:
indents_left = np.array([x.h_left for x in data])
indents_right = np.array([x.h_right for x in data])

In [104]:
print(np.sort(indents_right)[::2])

[ 63  63  65  66  81  86  93  93  93  93  93  93  93  93  93  96  98 111
 112 126 128 142 148 153 157 161 163 163 164 167 167 170 174 179 183 184
 185 190 193 196 197 201 202 208 211 212 213 214 220 223 226 226 226 228
 229 234 235 236 236 240 241 242 248 249 251 252 252 255 256 259 259 261
 261 262 264 264 266 267 267 268 269 270 270 271 273 274 274 276 277 279
 282 282 283 283 286 287 287 289 289 290 291 294 294 296 298 298 298 298
 298 299 300 302 303 304 306 306 307 308 309 311 312 312 314 315 315 316
 316 318 319 319 319 319 320 320 321 321 322 322 322 323 324 324 325 326
 327 328 328 329 329 331 332 333 334 335 337 337 337 338 339 342 342 343
 343 345 346 349 352 352 352 353 355 358 361 365 366 367 369 370 371 372
 375 379 379 381 382 385 385 387 388 389 390 390 394 395 395 397 398 400
 404 405 406 407 410 412 413 413 416 419 421 422 426 429 431 432 433 434
 437 437 440 443 445 449 452 453 455 455 457 460 461 464 465 468 470 472
 476 479 480 484 484 489 491 492 497 500 502 503 50

Здесь можно для каждого журнала найти [и захардкодить] "важные" значения: отступы для основного текста, для начал абзацев, ...

Мы попытаемся определять эти границы при помощи некоторой статистики.

In [105]:
margin_lefts = np.array([x.h_left for x in data])

unique_margin_lefts, pos_margin_lefts = np.unique(margin_lefts, return_inverse=True)
counts_margin_lefts = np.bincount(pos_margin_lefts)
sorted_margin_lefts = np.argsort(counts_margin_lefts)[::-1]


margin_rights = np.array([x.h_right for x in data])

unique_margin_rights, pos_margin_rights = np.unique(margin_rights, return_inverse=True)
counts_margin_rights = np.bincount(pos_margin_rights)
sorted_margin_rights = np.argsort(counts_margin_rights)[::-1]

## Ищем предполагаемые границы абзацев.

In [106]:
EPS = 4

left_keys = list()
left_indents = defaultdict(int)
for a, b in zip(unique_margin_lefts[sorted_margin_lefts[:15]], counts_margin_lefts[sorted_margin_lefts[:15]]):
    index = -1
    print(a, b)
    for l in range(a - EPS, a + EPS + 1):
        if l in left_indents:
            index = l
    if index != -1:
        left_indents[index] += b
    else:
        left_indents[a] += b
        left_keys.append(a)

left_len = 1
left_sum = left_indents[left_keys[0]]
while True:
    if left_indents[left_keys[left_len]] * 1.2 >= left_indents[left_keys[left_len - 1]]:
        left_sum += left_indents[left_keys[left_len]]
        left_len += 1
    else:
        break
        
print(left_len)


right_keys = list()
right_indents = defaultdict(int)
for a, b in zip(unique_margin_rights[sorted_margin_rights[:15]], counts_margin_rights[sorted_margin_rights[:15]]):
    index = -1
    print(a, b)
    for l in range(a - EPS, a + EPS + 1):
        if l in right_indents:
            index = l
    if index != -1:
        right_indents[index] += b
    else:
        right_indents[a] += b
        right_keys.append(a)

right_len = 1
right_sum = right_indents[right_keys[0]]
while True:
    if right_indents[right_keys[right_len]] * 1.2 >= right_indents[right_keys[right_len - 1]]:
        right_sum += right_indents[right_keys[left_len]]
        right_len += 1
    else:
        break
        
print(right_len)

62 355
90 162
83 67
98 19
72 19
317 15
80 12
66 6
331 6
296 6
313 6
309 6
262 5
310 5
265 5
1
533 241
532 113
534 28
93 18
503 13
298 10
529 10
504 8
319 8
535 7
526 7
528 6
516 6
352 6
322 5
1


In [107]:
assert left_len == right_len

In [108]:
left_keys, right_keys

([62, 90, 83, 98, 72, 317, 331, 296, 309, 262],
 [533, 93, 503, 298, 319, 526, 516, 352])

In [109]:
LEFT_BORDERS = np.sort(left_keys[:left_len])
RIGHT_BORDERS = np.sort(right_keys[:right_len])

In [110]:
LEFT_BORDERS, RIGHT_BORDERS

(array([62]), array([533]))

In [111]:
AVG_PARAGRAPH_LEN = np.mean(RIGHT_BORDERS - LEFT_BORDERS)

In [112]:
AVG_PARAGRAPH_LEN

471.0

In [113]:
LEFT_MAGRINS = unique_margin_lefts[sorted_margin_lefts[len(LEFT_BORDERS):2 * len(LEFT_BORDERS)]]

In [114]:
LEFT_MAGRINS

array([90])

# Определение шрифта основного текста

Это должен быть самый частый шрифт для строк с основным текстом.

In [115]:
font_names = defaultdict(int)

REGULAR_FONT_NAME = None
REGULAR_FONT_SIZE = None
max_font_name = 0

for x in data:
    if min(abs(x.h_left - LEFT_BORDERS)) <= EPS and min(abs(x.h_right - RIGHT_BORDERS)) <= EPS:
        # чтобы уж наверняка, выбираем только полные строки
        font_names[(x.font0_name, x.font0_size)] += 1
        font_names[(x.font1_name, x.font1_size)] += 1
        if font_names[(x.font0_name, x.font0_size)] > max_font_name:
            max_font_name = font_names[(x.font0_name, x.font0_size)]
            REGULAR_FONT_NAME = x.font0_name
            REGULAR_FONT_SIZE = x.font0_size
        if font_names[(x.font1_name, x.font1_size)] > max_font_name:
            max_font_name = font_names[(x.font1_name, x.font1_size)]
            REGULAR_FONT_NAME = x.font1_name
            REGULAR_FONT_SIZE = x.font1_size
        

In [116]:
REGULAR_FONT_NAME, REGULAR_FONT_SIZE

('g_d0_f1', 12)

# Процессинг документа (первичный)

In [117]:
class State(Enum):
    NEUTRAL = 0
    PARAGRAPH = 1
    BUKVITSA = 2

class Label(Enum):
    PAR_STARTS = 'A'
    PAR_CONTINUES = 'B'
    HEADER = 'C'
    BUKVITSA = 'D'
    LIST = 'E'
    OTHER = '?'

cur_state = State["NEUTRAL"]

In [118]:
EPS = 4
EPS_FONT = 2

In [119]:
def process_line(line, cur_state):
    # ищем колонку, куда помещается строчка
    ind_block = -1
    for i in range(len(LEFT_BORDERS)):
        l = LEFT_BORDERS[i]
        r = RIGHT_BORDERS[i]
        if l - EPS <= line.h_left and line.h_right <= r + EPS:
            ind_block = i
    
    # если колонка не обнаружена, выходим
    if ind_block == -1:
        return Label["OTHER"], State["NEUTRAL"]
    
    # иначе колонка найдена

    
    is_font0_same = abs(line.font0_size - REGULAR_FONT_SIZE) <= EPS_FONT
    is_font1_same = abs(line.font1_size - REGULAR_FONT_SIZE) <= EPS_FONT
    
    is_font0_regular = line.font0_name == REGULAR_FONT_NAME
    is_font1_regular = line.font1_name == REGULAR_FONT_NAME
    
    is_aligned_left = abs(LEFT_BORDERS[ind_block] - line.h_left) <= EPS
    is_aligned_right = abs(RIGHT_BORDERS[ind_block] - line.h_right) <= EPS
    
    bukvitsa_size = 4000
    
    if is_aligned_left and is_aligned_right:
        if is_font0_same or is_font1_same:
            return Label["PAR_CONTINUES"], State["PARAGRAPH"]
        else:
            return Label["OTHER"], State["NEUTRAL"]
    elif is_aligned_left:
        # конец абзаца
        if is_font0_same or is_font1_same:
            return Label["PAR_CONTINUES"], State["PARAGRAPH"]
        # буквица
        elif abs(line.font0_size - bukvitsa_size) <= EPS or abs(line.font1_size - bukvitsa_size) <= EPS:
            return Label["BUKVITSA"], State["BUKVITSA"]
        # заголовок
        else:
            if (not is_font0_regular) or (not is_font1_regular):
                return Label["HEADER"], State["NEUTRAL"]
            else:
                return Label["OTHER"], State["NEUTRAL"]
    elif is_aligned_right:
        if cur_state == State["BUKVITSA"]:
            return Label["PAR_CONTINUES"], State["PARAGRAPH"]
        elif is_font0_same or is_font1_same:
            tokens = line.text.split()
            if len(tokens) > 0:
                first_token = tokens[0]
                filtered = ''.join(list(filter(lambda x: x in 'ЁЙЦУКЕНГШЩЗХЪФЫВАПРОЛДЖЭЯЧСМИТЬБЮ', first_token)))
                if len(filtered) > 0:
                    return Label["PAR_STARTS"], State["PARAGRAPH"]
                else:
                    return Label["LIST"], State["PARAGRAPH"]
            else:
                return Label["OTHER"], State["NEUTRAL"]
        # заголовок
        else:
            return Label["HEADER"], State["NEUTRAL"]
    else:
        if (not is_font0_regular) and (not is_font1_regular):
            return Label["HEADER"], State["NEUTRAL"]
        else:
            tokens = line.text.split()
            if len(tokens) > 0:
                first_token = tokens[0]
                filtered = ''.join(list(filter(lambda x: x in 'ЁЙЦУКЕНГШЩЗХЪФЫВАПРОЛДЖЭЯЧСМИТЬБЮ', first_token)))
                if len(filtered) > 0:
                    return Label["OTHER"], State["NEUTRAL"]
                else:
                    return Label["LIST"], State["PARAGRAPH"]
            else:
                return Label["OTHER"], State["NEUTRAL"]

# Процессинг документа (вторичный): чистка заголовков
После заголовка должен идти абзац.

In [120]:
Label["OTHER"].value

'?'

In [121]:
def write_labeled(output_file_name=None):
    if output_file_name is None:
        output_file_name = structure_file_name[:-5] + '_l' + structure_file_name[-5:]
    with open(output_file_name, 'w', encoding='utf8') as output:
        cur_state = State["NEUTRAL"]
        
        PAR_INDENTS = defaultdict(int)
        
        for line in data:
            label, cur_state = process_line(line, cur_state)
            line.label = label.value
            if label == Label["PAR_STARTS"]:
                index = line.h_left
                for l in range(line.h_left - EPS, line.h_left + EPS + 1):
                    if l in PAR_INDENTS:
                        index = l
                PAR_INDENTS[index] += 1
        
        #print(PAR_INDENTS)
        
        for line in data:
            if line.label == Label["OTHER"].value:
                #print('YES', line.text)
                index = line.h_left
                for l in range(line.h_left - EPS, line.h_left + EPS + 1):
                    if l in PAR_INDENTS:
                        index = l
                #print(index)
                if PAR_INDENTS[index] > 1:
                    line.label = Label["PAR_STARTS"].value
        
        
        #bukv_found = False
        #for i in range(len(data) - 1, 0, -1):
        #    if bukv_found:
        #        data[i].label = Label["OTHER"]
        #    elif data[i].label == Label["BUKVITSA"]:
        #        bukv_found = True
        #    elif data[i - 1].label == Label["HEADER"]:
        #        if data[i].label not in [Label["PAR_STARTS"], Label["HEADER"]]:
        #            data[i - 1].label = Label["OTHER"]
        #        elif data[i].label == Label["HEADER"]:
        #            data[i].font0_name != 
            
            
        
        for line in data:
            line_export = [line.page_number, line.line_number, line.first_symbol, line.int_from_prev, 
                           line.int_from_next, line.h_left, line.h_right, line.v,
                           line.font0_size, line.font0_name, line.font1_size, line.font1_name,
                           line.label, line.text]

            line_export = ':'.join([str(y) for y in line_export])
            output.write(f'{line_export}\n')

In [122]:
write_labeled()

In [123]:
import io
from reportlab.pdfgen import canvas
from PyPDF2 import PdfFileWriter, PdfFileReader

In [124]:
def draw_rectaingles(lines, input_file_name, output_file_name=None):
    if output_file_name is None:
        output_file_name = structure_file_name[:-4] + '_l' + structure_file_name[-4:]
    
    color_dict = {'A': (1.0, 0.0, 0.0),
                  'B': (0.0, 0.1, 0.0),
                  'C': (0.0, 0.0, 1.0),
                  'D': (1.0, 0.0, 1.0),
                  'E': (1.0, 1.0, 0.0),
                  '?': (0.5, 0.5, 0.5),
                 }
    input_file = PdfFileReader(open(input_file_name, "rb"))
    output_file = PdfFileWriter()
    cur_page = 0
    input_page = input_file.getPage(cur_page)
    
    for line in lines:
        print(line.label, line.text)
        c = canvas.Canvas('watermark.pdf')
        c.setStrokeColorRGB(*color_dict[line.label])
        c.rect(line.h_left, line.v, line.h_right - line.h_left, 8, stroke=1, fill=0)
        c.save()
    
        watermark = PdfFileReader(open("watermark.pdf", "rb"))    
        
        if line.page_number - 1 == cur_page:
            # new page == old page
            input_page.mergePage(watermark.getPage(0))
        else:
            # перешли на новую страницу
            output_file.addPage(input_page)
            cur_page = line.page_number - 1
            input_page = input_file.getPage(cur_page)
            input_page.mergePage(watermark.getPage(0))
        
    output_file.addPage(input_page)
        
    with open(output_file_name, "wb") as o:
        output_file.write(o)

In [125]:
draw_rectaingles(data, 'processed/aerospace/y:2015:i:1:p:31-57.pdf')

C  
C Ссылка на статью: 
C // Аэрокосмический научный журнал.  
? МГТУ им. Н.Э. Баумана. Электрон. журн. 2015. 
E No 01. С. 31–57. 
C DOI: 10.7463/aersp.0115.0775178 
? Представлена в редакцию: 12.01.2015 
C  
C © МГТУ им. Н.Э. Баумана 
B УДК 543.8 + 541.13 
C Истечение вязкой жидкости через круглые 
C отверстия при малых числах Рейнольдса 
B Пильгунов В. Н.
C 1,*
C , Ефремова К. Д.
C 1
?   
B  
A 1МГТУ им. Н.Э. Баумана, Москва, Россия 
?  
C  
A Рассмотрена  гидродинамика  и  исследован  процесс  истечения  минерального  масла через 
E круглое отверстие диаметром 0,9 мм с острой входной кромкой, расположенное в удлинённой 
E цилиндрической  камере  диаметром  20  мм.  В  процессе  эксперимента  обнаружен  срыв 
E ламинарного  режима истечения  вязкой  жидкости  при  малом  числе  Рейнольдса  Re  =  430, 
E обусловленный  кавитацией  на  острой  кромке  отверстия.  При  числах  Рейнольдса  Re  >  500 
E начинается  интенсивное  пенообразование  в  свободной  струе,  обусловленное  кром

B режиме (без  выделения парогазовых  пузырьков),  то  смесь  имеет  плотность  однофазной 
B жидкости  см =  и коэффициент массового расхода (5) примет вид 
C       
C   
C  
C          
C  
C   
E  . (6)  
C Б. Затопленная струя 
A Жидкость истекает через отверстие в режиме затопленной струи, если пропускная 
B способность  отвода В меньше  расхода  струи,  а  выходная  камера  изолирована  от 
B атмосферы (верхняя половина рис. 1). В этом случае уравнение Бернулли, записанное для 
B сечений 1 – 2 в избыточной системе единиц измерения давления, имеет вид 
C  
C   
C   
C   
C   
C   
C   
C     
C   
C     
C   
C   
E             , (7)  
B где v2 = vк – осредненная  скорость  движения  жидкости  в  сечениях  подводящего  и 
B отводящего каналов камеры; 
E  р  1 – коэффициент  гидравлических  потерь  при  расширении  потока  в  формуле 
B Борда. 
A Выражая, как и прежде, скорости в уравнении (7) через массовый расход G, получим 
C  
C      
C  
C           
C  
C      
C  
C  
C  

B смеси  выделяются  из  жидкости.  Если  предположить,  что  объем  газа,  находящийся  в 
B ограниченном  объеме  двухфазной  смеси  (минерального  масла  с  нерастворенным 
B воздухом),  полностью  перейдет  в  растворенное  и  мелкодисперсное  нерастворенное 
B состояние, то можно оценить процент газосодержания жидкости в ее исходном состоянии 
B при атмосферном давлении, путем анализа соотношения объемов: 
C      
C      
E         . (12) 
A Здесь Wсм – первоначальный  объем  смеси,  измеряемый  в  течение  10  сек  после 
B наполнения  мерного  цилиндра; W – объем  минерального  масла  после  полного 
B растворения  мелкодисперсных  пузырьков  воздуха и  конденсации  паров  (определяется 
B через 30 минут после замера первоначального объема двухфазной смеси).  
A Результаты анализа показали, что при величине давления истечения p1  1800 кПа 
B можно принять Г = 6% = 0,06, а при p1  2000  кПа, Г = 7,5% = 0,075. Таким образом, 
B объем  минерального  масла  W  после полного  раств

B конвективных  ускорений V/S  0, обусловленных  изменением  направления  вектора 
B скорости (рис. 9).  
C  
C   
C Рис. 9. Схема огибания периферийной струйкой острой входной кромки.  
C  
A В  этом  случае   ускорение  направлено  к  центру  окружности  радиуса r 
B перпендикулярно    вектору    скорости    и    рассматривается    как    нормальное 
B центростремительное ускорение переносного движения 
C       
C   
C  
E  . (19) 
A На  частицу  жидкости  массой m,  расположенную  на  периферийной  струйке  и 
B огибающую  острую  кромку,  в  центре  ее  массы  действует  центробежная  сила    , 
B направленная от центра окружности 
C       
C      
C  
E  . (20)  
A Удерживает   частицу   на   траектории   ее   движения   уравновешивающая 
B центростремительная сила Fцс, определяемая силами давления со стороны соседствующих 
B с ней частиц с внешней стороны траектории (подпирающее усилие), и силами растяжения 
B со  стороны  частиц  с  внутренней  стороны  траектории,  прилипши

B сигналов использовались  электронно-лучевой  осциллограф  С1-83  и  анализатор  спектра 
B Я40-0830. 
A В  исследованиях  [2,  3]  показано  наличие  в кавитирующем потоке  минерального 
B масла  интенсивного  светоизлучения  в  голубой  части  спектра.  При  истечении  жидкости 
B через  круглое  отверстие  малого  диаметра,  выполненное  в  стальное  пластине, 
B светоизлучение на острой кромке не наблюдалось.  
A Одной  из  возможных  причин  отсутствия  светоизлучения  при  наличии  активных 
B электрических  процессов  в  сжатом  сечении  струи  можно  считать  интенсивное 
B пенообразование, изменившее условия, приводящие к светоизлучению.  
A В  упомянутых  выше  работах  [2...5]  диафрагменный  дроссель  с  условно  острой 
B кромкой  (900)  был  выполнен  из  диэлектрика,  поэтому  на кромке мог  накапливаться 
B электрический потенциал, который инициировал светоизлучение.  
A На  металлической  шайбе  электрический  потенциал  может  быть  рассредоточен  в 
B объеме  металл

E 2015. No 2. С. 1-23. DOI: 10.7463/0215.0758817     
E 2. Пильгунов  В.Н.,  Ефремова  К.Д.  Светоизлучение  и  электрические  процессы  в 
E кавитирующем потоке  минерального масла //  Наука и образование. МГТУ им.  Н.Э. 
? Баумана. Электрон. журн. 2013. No 3. С. 31-62. DOI: 10.7463/0313.0535547      
E 3. Маргулис   М.А.,   Пильгунов   В.Н.   Свечение  и   электризация  при   течении 
E диэлектрических жидкостей в узком канале // Журнал физической химии. 2009. Т. 83, 
E No 8. С. 1585-1590.    
E 4. Маргулис М.А., Пильгунов В.Н. О механизме возникновения свечения и электризации 
E при течении жидкостей в узком канале // Журнал  физической химии. 2009. Т. 83, No 
E 10. С. 1975-1979.     
E 5. Маргулис М.А., Пильгунов В.Н. Механизм свечения и электризации жидкостей при 
E течении в узких каналах // Вестник МГТУ им. Н.Э. Баумана. Сер. Естественные 
E науки. 2010. No 1. С. 64-79. Режим доступа: 
C http://vestniken.bmstu.ru/catalog/phys/hidden/129.html  (дата обращения 01.12.2014).    
E 6

E N.E. Baumana = Science and Education of the Bauman MSTU, 2012, no. 5, pp. 1-17. DOI: 
C 10.7463/0512.0370692     (in Russian). 
B 10. Deo R.C. Comparative Analysis of Turbulent Plane Jets from a Sharp-Edged Orifice, a Bev-
E eled-Edge Orifice and  a  Radially Contoured Nozzle. Engineering and Technology Interna-
E tional  Journal  of  Mechanical,  Aerospace,  Industrial  and  Mechatronics  Engineering,  2013, 
C vol. 7, no. 12, pp. 1496-1505. 
B 11. De Giorgi M.G., Ficarella A., Tarantino M. Evaluating cavitation regimes in an internal ori-
C fice at different temperatures using frequency analysis and visualization. International Jour-
C nal of Heat and Fluid Flow, 2013, vol. 39, pp. 160-172. DOI: 
C 10.1016/j.ijheatfluidflow.2012.11.002   


In [ ]:
data[0].label